In [ ]:
# autoreload
%load_ext autoreload
%autoreload 2

from networks.novel.lite_vae.blocks.haar_own import HaarTransform
import torch
# from networks.novel.lite_vae.blocks.smc         import SMC
# from networks.novel.lite_vae.blocks.resblock    import ResBlock, ResBlockWithSMC
# from networks.novel.lite_vae.blocks.midblock    import MidBlock2D
# from networks.novel.lite_vae.blocks.unet_block  import LiteVAEUNetBlock
from networks.novel.lite_vae.encoder            import LiteVAEEncoder
from networks.novel.lite_vae.decoder            import *  # or SDVAEDecoder
from networks.novel.lite_vae.litevae            import LiteVAE


import torch
import torch.nn as nn
from torchinfo import summary
import numpy as np
# from typing import Dict, List
# import matplotlib.pyplot as plt
from data.image_dataset import *
from torch.utils.data import DataLoader
# from PIL import Image
# import os
import ptwt, pywt

In [ ]:
# Simulate grayscale images
haar = HaarTransform(levels=3)

# img_1 = Image.open(os.path.join('/media/ee/DATA/Talha_Nehal/Diffusion-Codes/GMS/Dataset/busi/images/benign_1.png')).convert("RGB")
# img_2 = Image.open(os.path.join('/media/ee/DATA/Talha_Nehal/Diffusion-Codes/GMS/Dataset/busi/images/benign_2.png')).convert("RGB")

# # stack them on the batch dimension and convert to tensor (B, C, H, W)
# img = torch.stack([torch.tensor(np.array(img_1).transpose(2, 0, 1)), 
#                    torch.tensor(np.array(img_2).transpose(2, 0, 1))])
# img.shape

train_dataset = Image_Dataset('Dataset/busi/busi_train_test_names.pkl', stage="train", excel = False)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=24,
    pin_memory=True,
    drop_last=True,
    shuffle=True,
)


In [ ]:
batch = next(iter(train_dataloader))


In [ ]:
img_batch = batch['img']
img_batch.shape

In [ ]:
recon, haar_transform_out_list, haar_transform_out_dict = haar(img_batch, inverse = True, stacked_version=True, visualize = True)

In [ ]:
wavelet=pywt.Wavelet('haar')
def dwt(x, level=None):

    level = level or level
    x_low, *x_high = ptwt.wavedec2(
    x.float(),
    wavelet=wavelet,
    level=level,
    mode='symmetric',
    )
    x_combined = torch.cat(
    [x_low, x_high[0][0], x_high[0][1], x_high[0][2]], dim=1
    )
    return x_combined
    
def idwt(self, x):
    x_low, x_high = x[:, :3], x[:, 3:]
    x_high = torch.chunk(x_high, 3, dim=1)
    x_recon = ptwt.waverec2([x_low.float(), x_high.float()], wavelet=wavelet)
    return x_recon

In [ ]:
# compute MSE of reconstructed image and original image
mse = torch.nn.functional.mse_loss(recon, img_batch)
print(f'MSE: {mse.item()}')

In [ ]:
mse * 224 * 224 * 3

In [ ]:
recon[0, :].unsqueeze(0).shape

In [ ]:
dwt_L1 = dwt(img_batch, level=1) / 2
dwt_L2 = dwt(img_batch, level=2) / 4
dwt_L3 = dwt(img_batch, level=3) / 8

In [ ]:
dwt_L1.shape, dwt_L2.shape, dwt_L3.shape

In [ ]:
x_low, x_high, wavelet_recon = dwt(recon, level = 1)
wavelet_recon = wavelet_recon / 2

In [ ]:
x_low.shape

In [ ]:
wavelet_recon.shape

In [ ]:
haar_transform_out_list[0].shape

In [ ]:
# compute MSE of reconstructed image and original image
mse = torch.nn.functional.mse_loss(wavelet_recon, haar_transform_out_list[0])
print(f'MSE: {mse.item()}')

In [ ]:
class CharbonnierLoss(nn.Module):
    def __init__(self, epsilon=1e-3):
        super(CharbonnierLoss, self).__init__()
        self.epsilon = epsilon

    def forward(self, prediction, target):
        diff = prediction - target
        loss = torch.mean(torch.sqrt(diff * diff + self.epsilon**2))
        return loss

criterion = CharbonnierLoss(epsilon=1e-3)
loss = criterion(wavelet_recon, haar_transform_out_list[0])
# print(f"Charbonnier Loss: {loss.item()}")

In [ ]:
loss

In [ ]:
from diffusers import AutoencoderTiny

vae = AutoencoderTiny.from_pretrained("madebyollin/taesd", torch_dtype=torch.float32, device_map = 'cpu')

In [ ]:
decoder = vae.decoder

In [ ]:
# Freeze decoder and count the number of learnable parameters before and after

learnable_params_before = sum(p.numel() for p in decoder.parameters() if p.requires_grad)
for param in decoder.parameters():
    param.requires_grad = False
learnable_params_after = sum(p.numel() for p in decoder.parameters() if p.requires_grad)

In [ ]:
learnable_params_before, learnable_params_after

In [ ]:
summary(decoder, input_size = (2, 4, 28, 28), depth = 2, col_names = ("kernel_size", "num_params"))

In [ ]:
base_model = LiteVAE(encoder = LiteVAEEncoder(), decoder = decoder)

In [ ]:
latent_batch = base_model(img_batch)

In [ ]:
latent_batch['latent'].shape, latent_batch['latent'].device, latent_batch['latent'].dtype

In [ ]:
summary(base_model, input_size = (2, 3, 224, 224), depth = 2, col_names = ("kernel_size", "num_params"))